In [1]:
import numpy as np
import scipy as sp
import sympy as sy
import json 
import h5py
import math

import SimulatorParser as sim_par
import SimulatorRater as sim_rater

In [2]:

###* Input files

reactions_file = "reactionsSimple.json"

const_dict = {
        "F0": 1.5e15,           # cm^-2
        "S0": 3e13,             # cm^-2
        
        "R": 0.00831442,        # kJ/mol*K
        "kBoltz": 1.380649e-23, # J/K
}

exp_data_file = "Experimental_data_Paper.hdf5"

In [3]:

###* call the classes 

parser = sim_par.SimulatorParser(reactions_file, const_dict)
output_parser = parser.create_physical_model()

simRates = sim_rater.SimulatorRates(reactions_file, const_dict, exp_data_file, output_parser)

exp_vec_arr, _ = simRates.prepare_experimental_data()
simRates.preload_rates_functions()


dO_F/dt = -0.02*O_F*O_S*r_7 - 1.0*O_F*r_2 - 1.0*O_F*r_4 - 0.02*O_F*r_5*(1.0 - O_S) + 1.0*r_1*(1.0 - O_F)
dO_S/dt = -1.0*O_F*O_S*r_7 + 1.0*O_F*r_5*(1.0 - O_S) - 1.0*O_S*r_6 + 1.0*r_3*(1.0 - O_S)


In [4]:


rates_calculation_arr = simRates.compute_rates_simulation(exp_vec_arr)

print(rates_calculation_arr[:3])

[[1.43876053e+04 6.45547732e+08 1.43876053e+04 1.43876053e+04
  6.02594888e+09 1.49125257e+01 6.00136101e+05]
 [2.40428746e+04 6.45547732e+08 2.40428746e+04 2.40428746e+04
  6.02594888e+09 2.69037303e+01 6.00136101e+05]
 [2.96392447e+04 6.45547732e+08 2.96392447e+04 2.96392447e+04
  6.02594888e+09 3.55964732e+01 6.00136101e+05]]


In [5]:

#####! modify the rates for the model

mod_model_dict = [
    (1, {'SF': 0.5, 'E': 1.0}),
    (5, {'SF': 0.5, 'E': 10.0, 'nu_D': 2e13}),
    (7, {'nu_D': 3e13})
]


rates_calculation_arr_2 = simRates.compute_rates_simulation(exp_vec_arr, mod_model_dict)

print(rates_calculation_arr_2[:3])

[[4.54983903e+03 6.45547732e+08 1.43876053e+04 1.43876053e+04
  6.48192572e+10 1.49125257e+01 1.80040830e+06]
 [7.64207807e+03 6.45547732e+08 2.40428746e+04 2.40428746e+04
  6.48192572e+10 2.69037303e+01 1.80040830e+06]
 [9.46541790e+03 6.45547732e+08 2.96392447e+04 2.96392447e+04
  6.48192572e+10 3.55964732e+01 1.80040830e+06]]


In [7]:

# mod_model_dict = [
#     (1, {'SF': 0.5, 'E': 1.0}),
#     (5, {'SF': 0.5, 'E': 10.0, 'nu_D': 2e13}),
#     (7, {'nu_D': 3e13})
# ]

# mod_model_map = {rate_id: params for rate_id, params in mod_model_dict }

# # mod_model_map = {}

# for reaction in output_parser['reactions_list']:
#     model_dict = reaction['model_dict']
#     rate_id   = reaction['id']
    
#     params_to_apply = mod_model_map.get(rate_id)
#     if params_to_apply is not None:
#         model_dict.update(params_to_apply)

#     print(model_dict)
#     print()

In [ ]:

model_simulator = output_parser['model']

state_list = [0.5, 0.3]



print(model_simulator(state_list, rates_calculation_arr[-1]))


print(output_parser.keys())
print(output_parser['species_model'])

initial_state_dict = {'O_F':0.1, 'O_S':0.5}

state_list = [initial_state_dict[ele] for ele in output_parser['species_model']]
print(state_list)



### parser - done
### rates  - done

###* modify rates for oprtimization
###* function for calling the solver - almost done
###* compute gammas


def system_ode(X, t, rates_vec):
    
    return model_simulator(X, rates_vec)



print(system_ode(state_list, 1.0, rates_calculation_arr[-1]))

[-7018361433788.213, 349435719703139.9]
dict_keys(['model', 'species_model', 'rates_model', 'reactions_list'])
['O_F', 'O_S']
[0.1, 0.5]
[-1005177034210.9801, 49811906567714.89]


In [ ]:

#### init_tuple must go with a dicitionary

import time


def solution_check(sol, rates_vec):
    
    vec_aux = system_ode(sol, 0.0, rates_vec)
    absolute_error = np.sum(np.abs(vec_aux))
    
    if absolute_error > 1e-4:
        return False
    else:
        return True


timeSpace = np.linspace(0, 1_00, 5_00)

short_time = np.linspace(timeSpace[0], timeSpace[min(30, len(timeSpace)-1)], 30)

max_time = 15
start_time = time.time()
events = None
if max_time is not None:
    def timeout_event(t, X):
        elapsed = time.time() - start_time
        return max_time - elapsed
            
    timeout_event.terminal = True
    timeout_event.direction = -1
    events = [timeout_event]
        
    sol_short = sp.integrate.solve_ivp(
        fun=lambda t, X: system_ode(X, t, rates_calculation_arr[-1]),
        t_span=(short_time[0], short_time[-1]),
        y0=state_list,
        method="Radau",
        t_eval=short_time,
        atol=1e-5, rtol=1e-5,
        events=events
    )
    refined_guess = sol_short.y.T[-1]

    print(refined_guess)

    ### Attempt to find the fixed point using the refined guess
    try:
        sol = sp.optimize.root(system_ode, refined_guess, args=(0, rates_calculation_arr[-1]), method="hybr")
        success = solution_check(sol.x, rates_calculation_arr[-1])
    except Exception as e:
        print("Fixed point solver failed with message: ", e)
        success = False
        # sol = self.init_conditions
    
    print(sol)
    print(success)

[3.78428887e-11 9.96250846e-01]
 message: The solution converged.
 success: True
  status: 1
     fun: [-6.217e-15  3.351e-13]
       x: [ 3.784e-11  9.963e-01]
  method: hybr
    nfev: 4
    fjac: [[-1.000e+00  8.467e-03]
           [-8.467e-03 -1.000e+00]]
       r: [ 1.641e+11 -1.076e+03  3.798e+04]
     qtf: [ 9.055e-15 -3.351e-13]
True


In [ ]:

###* solve with ode method


timeSpace = np.linspace(0, 10.0, 1_00)
solution = sp.integrate.odeint(
    func=lambda X, t: system_ode(X, t, rates_calculation_arr[-1]),
    y0=state_list,
    t=timeSpace
)


sol2 = sp.integrate.solve_ivp(
    fun=lambda t, X: system_ode(X, t, rates_calculation_arr[-1]),
    t_span=(timeSpace[0], timeSpace[-1]),
    y0=state_list,
    method="BDF",
    t_eval=timeSpace, 
    atol=1e-5, rtol=1e-5
)

solution2 = sol2.y.T

print(solution[-1])
print(solution2[-1])

[3.78428887e-11 9.96250846e-01]
[3.78428887e-11 9.96250846e-01]


In [ ]:

####* compute gammas


reaction_list = output_parser['reactions_list']
rates_list = rates_calculation_arr[-1]
steady_solution_dict = {ele:solution2[-1,idx] for idx, ele in enumerate(output_parser['species_model'])}

exp_data_list = exp_vec_arr[-1]

print(steady_solution_dict)
print(exp_data_list)


idx_gamma_vec = [idx for idx, r in enumerate(reaction_list) if r['gamma']]

# print(idx_gamma_vec)
# print(rates_list)
# print(rates_list[idx_gamma_vec])
# print(reaction_list)


gammas_result = dict()

for idx in idx_gamma_vec:
    
    if reaction_list[idx]['gas_specie'] == 'O2':
        factor = 1.0
    else:
        factor = 2.0
    
    species_involved_list = list(reaction_list[idx]['left'].keys())
    family_species_list = [ele.split('_')[1] for ele in species_involved_list]
    
    print(idx)
    print(factor)
    print(species_involved_list)
    print(family_species_list)
    
    flag_S = 'S' in family_species_list
    
    frac_list = [steady_solution_dict[ele] for ele in species_involved_list]
    value_gamma = factor * rates_list[idx] * math.prod(frac_list) *(const_dict['S0'] if flag_S else const_dict['F0']) / exp_data_list['O']
    
    print((const_dict['S0'] if flag_S else const_dict['F0']))
    print()
    
    gammas_result["r_" + str(reaction_list[idx]['id'])] = value_gamma


print(gammas_result)

gamma_total = sum(gammas_result.values())
print(gamma_total)




{'O_F': 3.7842888749771973e-11, 'O_S': 0.9962508456959104}
{'pressure': 7.5, 'current': 40.0, 'Tnw': 409.91221146264, 'Tw': 323.15, 'O': 9500750822102000.0, 'CO': 0.0, 'N': 1.14464276533369e+17, 'Ion': 4000000000000000.0, 'EavgMB': 0.001}
3
2.0
['O_F']
['F']
1500000000000000.0

5
2.0
['O_S']
['S']
30000000000000.0

6
2.0
['O_F', 'O_S']
['F', 'S']
30000000000000.0

{'r_4': 9.09932004752347e-13, 'r_6': 0.00047909689730848737, 'r_7': 0.8956739301303085}
0.896153027028527
